In [333]:
!pip install BeautifulSoup4
!pip install requests



In [348]:

class Advert:
    def __init__(self, id: str, title: str, price: int, reg_year_month: str, body_type: str, miles: int, cc: float, transmission: str, fuel: str, url: url):
        self.id = id
        self.title = title
        self.price = price
        self.reg_year_month = reg_year_month
        self.body_type = body_type
        self.miles = miles
        self.cc = cc
        self.transmission = transmission
        self.fuel = fuel
        self.url = url 
        self.descriptionRaw = ""
        self.other1 = ""
        self.other2 = ""
        self.other3 = ""
        
        
import requests
from bs4 import BeautifulSoup

spec_list = []

In [349]:
print("345")

## initialise
car_count = 0

page_count = 0
page = 1
site_domain = "https://www.autotrader.co.uk"

search_url = "/car-search"
detail_url ="/classified/advert/"
query_param = "?sort=price-asc&radius=200&postcode=tn278eh&onesearchad=Used&make=AUDI&model=R8&year-from=2007&year-to=2019&body-type=Coupe&transmission=Automatic"
#query_param = "?sort=price-asc&radius=200&postcode=tn278eh&onesearchad=Used&make=AUDI&model=Q5&transmission=Automatic"
has_more_pages = True
while has_more_pages :
    # request a page
    url  =  site + search_url + query_param + "&page=" + str(page)
    print(url)
    response = requests.get( url )
    soup = BeautifulSoup(response.text, 'html.parser')

    # did we fi d any ad's
    
    car_count = int(soup.find('h1', class_="search-form__count").text.split(" ")[0])
    if len(spec_list) == car_count:
        print("PageCount: {}, SpecList: {}".format(page, len(spec_list)) )
        # No ad's found - stop
        break
        
        
    page_count = int( (car_count + 5) / 10 )

    print("Processing Ad's from page {} of {}. For total of Car count: {}".format(page, page_count, car_count) )
    adverts = soup.find_all('article', class_="search-listing") 

    # Process Ad's on current page
    ad_number = 0
    for advert in adverts :
        ad_number += 1
        price = advert.find('div', class_="vehicle-price").text.replace("£","").replace(",","")
        
        info_div = advert.find( 'div', class_="information-container")
        title = info_div.find( 'h2', class_="listing-title").text
        ad_url = info_div.find( 'a', class_="listing-fpa-link")['href']
        ad_id = ad_url.split("/")[3].split("?")[0]
        
        ## Check fro dups
        is_dup = False
        for exist_ad in spec_list:
            #print("id- {} dup check {} ".format(ad_id, str(exist_ad) ) )
            if exist_ad.id == str( ad_id):
                is_dup = True
                break
        if is_dup:
            print("skipping duplicate add for ad_id "+ ad_id)
            continue
            
        
        spec_tags = info_div.find( 'ul',  class_="listing-key-specs").findChildren()

        ##print("Page: {}, ad: {}, parse spec structure: {}".format(page, ad_number, spec_tags))
        raw_year = spec_tags[0].text.split(" ")
        raw_bodyType = spec_tags[1].text.lower()
        raw_miles = spec_tags[2].text.replace(",","").split(" ") 
        raw_cc = spec_tags[3].text.replace("L","")
        ## this is horrid
        ## 1 2 3 4 5 6
        ## 1 2 3 5 6       <- for some ad's the bhp was missing
        if ( spec_tags[4].text[-3:] == "bhp"):
            raw_bhp = spec_tags[4].text.split(" ")
            raw_transmission = spec_tags[5].text.lower()
            raw_fuel = spec_tags[6].text.lower()
        else:
            raw_bhp = ""
            raw_transmission = spec_tags[4].text.lower()
            raw_fuel = spec_tags[5].text.lower()
            
        spec_list.append( Advert(ad_id, title, int(price), raw_year[0]+raw_year[1].replace("(","/"), raw_bodyType, int(raw_miles[0]), float(raw_cc),  raw_transmission, raw_fuel, site+detail_url+ad_id) )

    
       
        
    if page == page_count :
        has_more_pages = False
    else:
        page += 1
    
    # end of page processing, next page?                   

# end of Site scrape
print("Found {} ad specs".format( len(spec_list)) )

print("<253.")

345
https://www.autotrader.co.uk/car-search?sort=price-asc&radius=200&postcode=tn278eh&onesearchad=Used&make=AUDI&model=R8&year-from=2007&year-to=2019&body-type=Coupe&transmission=Automatic&page=1
Processing Ad's from page 1 of 10. For total of Car count: 99
https://www.autotrader.co.uk/car-search?sort=price-asc&radius=200&postcode=tn278eh&onesearchad=Used&make=AUDI&model=R8&year-from=2007&year-to=2019&body-type=Coupe&transmission=Automatic&page=2
Processing Ad's from page 2 of 10. For total of Car count: 99
skipping duplicate add for ad_id 201910213547933
skipping duplicate add for ad_id 201910213547933
https://www.autotrader.co.uk/car-search?sort=price-asc&radius=200&postcode=tn278eh&onesearchad=Used&make=AUDI&model=R8&year-from=2007&year-to=2019&body-type=Coupe&transmission=Automatic&page=3
Processing Ad's from page 3 of 10. For total of Car count: 99
skipping duplicate add for ad_id 201906259388565
skipping duplicate add for ad_id 201909182373026
skipping duplicate add for ad_id 20

In [350]:

## get individual ads for car images and details
ad_number = 0
for advert in spec_list:
    ad_number += 1
    print("inspection of Advert {} of {} for more details".format(ad_number, len(spec_list)) )
    print("url: " + advert.url)
    car_detail_response = requests.get( advert.url )
    soup_detail = BeautifulSoup(car_detail_response.text, 'html.parser')
    ##print(soup_detail)
    
    #<button type="button" class="gallery-thumbs__placeholder"><img src="https://m.atcdn.co.uk/a/media/w100h75/8a84642bf4a24ee1887b101ad68b4ede.jpg" alt=" "></button>
    
    meta_description = soup_detail.find('meta',attrs={'name':'og:description'})
    advert.descriptionRaw = meta_description["content"]
    
    print("Meta detial: #",ad_number," id:",advert.id,"  desc:",advert.descriptionRaw)
        
    
    
        
    
    

inspection of Advert 1 of 99 for more details
url: https://www.autotrader.co.uk/classified/advert/201907260477164
Meta detial: # 1  id: 201907260477164   desc: Price: £54,995. Four wheel-drive, Audi parking system plus with rear camera, Contrast stitching - Titanium grey, Cruise control, Metallic - Suzuka grey, Metallic paint REF:C3CGC. Grey, Stunning low mileage example equipped with £5945 in options costing £99845 new in Suzuka grey metallic. Just serviced by Audi, the car has been serviced at 3664 miles, 5175 and 6336 miles. The car features LED headlights, advanced parking system with rear view camera, heated seats, Audi Music Interface, Nappa leather sports seats, bluetooth phone interface, cruise control, sat nav, front and rear parking sensors, voice control, LED daytime running lamps, Quattro 4WD, Sport mode, climate control and more. All keys and bookpack present. PROMOTORS WERE AWARDED A TOP 10 RANKING IN THE 2018 AUTOTRADER RETAILER AWARDS OUT OF OVER 6,500 NATIONAL RETAILER

Meta detial: # 10  id: 201908311717875   desc: Price: £34,500. NOYA MOTORS  is delighted to offer stunning  example of a first generation Audi R8, GREY COLOUR, Semi Automatic 4.2 Liters, 2dr coupe, 420 BHP.  it has the following. stunning grey   Beige Fine Nappa Leather, full AUDI service history last serviced april 2019  full carbon pack. BANG & OLUFSEN sound. sat nav heated seats mint condition alloy wheels. electronic spoiler. Electric windows< Air conditioning, Parking aid, CD player, sports seats Please visit our website for full specification and hi-resolution photographs.. to book a test drive please contact our friendly and helpful team same day test drive available!.. NOYA MOTORS is run by car lover owner which carries out individually sourced, best price on AutoTrader guaranteed, all cars come with  service history, hpi clear, 6 months warranty, with Unbeatable flexible finance deals available with same-day approval 0% deposit finance available, Our showroom is located at 12 

Meta detial: # 16  id: 201906098831502   desc: Price: £35,000. SERVICED TODAY, 07/10/2019!! 2009 TOP Spec 4.2 V8 R-Tronic WARRANTY .  Full Carbon Fibre Pack. 2009 Audi R8 4.2 V8 R-Tronic 420BHP. Serviced this week!  65k Miles.12 Month 3rd party 5 STAR full warranty on all parts!!  Thatcham cat5 Cobra Trak 5 Tracking System. Just been valeted and machine polished / Leather treated.  Full Audi Service History.  Had a new clutch in 2018. Personalised number plate ?R8 UOU? will come with the car.  4 Brand new Michelin Pilot tyres at £1400  White with full black Carbon Fibre Extended pack.  Magnetic Ride with 2 shocks recently replaced at over £1200 each.  Bang and Olufsen Audio. Sat Nav, Cruise Control. Puddle Lights. Extended interior carbon fibre. Dash Cam. Disks have just been skimmed. This car is the obsolete top spec R8, not one extra is missing.  Car will come with an almost new Audi R8 ?Windforce? cover. This is designed for this car and fits perfect. It fully protects the car from 

Meta detial: # 23  id: 201904136950988   desc: Price: £39,950. HCS Trading Ltd is a small independent Family run business. We are FCA Authorised, and work with all major finance companies, we can arrange finance for you ready to  make your next purchase. We only charge an administration fee if finance is outsourced. We also offer part exchange and extendable 3rd party warranties.
All of our cars are HPI Clear and have a full service history. Our cars are prepared to a high standard and are fully valeted ready to be viewed. THE CAR HAS RECENTLY HAD CLUTCHES ON THE AUTOMATIC GEARBOX OVERHAUL OF THE REAR DIFF, INCLUDING THE LAST SERVICE IT HAS JUST HAD £8000 SPENT ON THE CAR, 10 MONTHS MOT
inspection of Advert 24 of 99 for more details
url: https://www.autotrader.co.uk/classified/advert/201906259388565
Meta detial: # 24  id: 201906259388565   desc: Price: £57,977. Driver's information system, DVD Satellite navigation system  plus + 6.5&quot; colour monitor with radio + MP3 player, PAS, Se

Meta detial: # 30  id: 201907140051556   desc: Price: £42,950. Four wheel-drive, Red, TRY US FIRST - WE WANT YOUR PART EXCHANGE SEE OUR WEBSITE FOR DETAILS, FINANCE AVAILABLE UP TO 5 YEARS WITH NO DEPOSIT!!! £10920 OF FACTORY FITTED OPTIONS! SUPER SPORT BUCKET SEATS & CARBON PACK - Main Dealer Service History 7 Services last Jul 19 at 71k, MOT till Jul 20, Tax Available, PAS, Climate, Cruise, Full Colour Sat Nav, Front & Rear Parking Aids, Bang & Olufsen CD Sound System, Heated Front Bucket Seats, Alcantara Multi-Function Steering Wheel with Paddleshift, Phone Prepared with Voice Control, Xenons with Daytime Running Lights, Privacy Glass, Carbon Fibre Dash Trim/Door Panels/Mirrors/Side Blade & Engine Bay Covers, Tracker, E/Windows, R/C/Locking, Alarm/Immobiliser, 19" Anthracite Twin Spoke Alloys, HPi Clear, Px Welcome, Fully Colour Coded in Brilliant Red Gloss with Full Black Nappa Leather Interior, EXCEPTIONAL MEGA SPEC'D R8 V10 QUATTRO AUTO IN GREAT COLOUR COMBO, FIRST TO SEE/DRIVE W

...

Meta detial: # 69  id: 201910032899848   desc: Price: £78,990. PRESTIGE CARS ARE DELIGHTED TO INTRODUCE THIS STUNNING R8 IN ARIA CRYSTAL BLUE TO THE MARKET BOASTING A GREAT SPEC AND HAS BEEN MAINTAINED WITH NO EXPENSE SPARED. FULL AUDI HISTORY.

APPOINTMENT ONLY***SAME DAY FINANCE DECISION 
inspection of Advert 70 of 99 for more details
url: https://www.autotrader.co.uk/classified/advert/201908020709976
Meta detial: # 70  id: 201908020709976   desc: Price: £79,950. Full Description:

Finished in Mythos Black Metallic with Black Diamond Stitched Fine Napa Leather Interior and Carpets creating a truly Stealthy Appearance. The car has done only 10k miles and is in perfect condition and running order. Full Audi Main Dealer Service History and a comprehensive specification, more of which follows below..


Full Specification includes:

Optional Extras Fitted to this vehicle:

20'' 10 Spoke 'Y' Design Gloss Black Alloy Wheels

Carbon Interior Pack

Driver Assistance Pack

Gloss Carbon Exterio

Meta detial: # 77  id: 201909092022280   desc: Price: £84,995. WE ARE DELIGHTED TO OFFER THIS STUNNING AUDI R8V10 PLUS, WORDS CAN NOT DESCRIBE HOW BEAUTIFUL THIS CAR IS AND IT HAS A HUGE AMOUNT OF SPECIFICATION. OPTIONS INCLUDE: SPORTS PLUS PACKAGE+ SOUND AND COMFORT PACKAGE+LED LAZER HEADLIGHTS+GLOSS CARBON ENGINE BAY TRIM+EXTENDED LEATHER PACKAGE FOR BUKET SEATS+20" 10 SPOKE ALLOY WHEELS+DRIVER ASSITANCE PACK+ALLUIMINIUM CARBON DOOR SILLS+BANG OULFSEN SPEAKER SYSTEM. WE OFFER COMPETITIVE FINANCE PACKAGES AND PART EXCHANGE IS MOST WELCOME. This car is VAT QUALYFING and this price includes VAT AND IS STILL UNDER MANUFACTURES WARRANTY.
inspection of Advert 78 of 99 for more details
url: https://www.autotrader.co.uk/classified/advert/201910253683551
Meta detial: # 78  id: 201910253683551   desc: Price: £86,900. DRIVER ASSIST PACK, B&O AUDIO, LED HEADLIGHTS WITH AUDI LASER LIGHT, REVERSE CAMERA Multi award-winning prestige car dealership John Holland is excited to offer for sale this outs

Meta detial: # 84  id: 201910233622134   desc: Price: £88,750. Car is in immaculate showroom condition and been looked after and cared for meticulously by a very proud owner.  Car was registered Sept 2016 on a 66 plate. Car brand new had over 17k of factory fitted extras so an absolute bargain for such a high specification car.  Finished in Vegas Yellow the car comes with so many standard and optional factory fitted options such as the LED Headlights With Audi Laser Light And High Beam Assist, Sound and Comfort pack to include Bang and Olufsen audio, illuminated door sill trims, Sport Plus Pack to include sport exhaust system, dynamic steering, Advanced Key, Exclusive Carbon Package, Audi Parking System Plus, R8 Sport Seats in Black Fine Nappa leather upholstery and trim with contrast diamond stitching design in Vegas yellow, Engine bay trim in Gloss Carbon, ISOFIX child seat mounting. Ceramic Brakes, Interior Lighting Pack, MMI Navigation Plus, MMI Touch, Audi Virtual Cockpit, DAB Dig

Meta detial: # 90  id: 201909172317598   desc: Price: £94,850. A confidence-inspiring supercar that's able to go to shops as well as the race track! This fabulous Audi R8 V10 Plus has been finished in Daytona Grey and is complemented by Diamond Stitched Grey Leather Sports Seats. This car has had just 1 former keeper and is presented for sale in absolutely stunning condition. It will be sold with the balance of its manufactures warranty. 

Sport Pack Plus, Sports Seats, 20" 10-Spoke Alloy Wheels, Driver Assistance Pack, Electric Front Seats, Fine Nappa Diamond Stitched Leather Seats.
Anti Locking Brakes, Driver and Passenger Airbags, Remote Central Locking with Alarm/Immobiliser, DAB Radio, Bluetooth Connectivity, HDD Navigation System, Audi Parking System, Start/Stop Function, Tyre Pressure Loss System, Electrically Adjustable Advanced Key System, Carbon Ceramic Brakes. 3 Spoke Flat Bottomed Sports Steering Wheel, Heated Front Seats, High-Beam Assist, Carbon Rear Spoiler, Climate Cont

Meta detial: # 98  id: 201908090956694   desc: Price: £103,000. Sound & Comfort Pack, Bang and Olufsen Sound system, 20" Gloss black 10 spoke Y design alloys, Sport Plus Pack, Audi Magnetic Ride, Sport exhaust system, Audi phone box, Cruise control, Reversing camera, Electric front seats with lumbar memory, Smart Phone Interface, High beam control, Dynamic steering, Front sports seats, Alcantara headlining, Garage door opener (HomeLink), Illuminated door sill trims, Matt Titanium Grey Sideblades, Audi Warranty 4yr 75000 miles from new, MMI navigation plus with DVD SD card reader, 10GB music storage, Audi Virtual Cockpit - 12.3" LCD instrument cluster, DAB Digital radio, Audi music interface, Audi parking system plus with front and rear sensors, Cruise control + speed limiter, Advanced key, Hill hold control, Deluxe climate contro. Black, Sound & Comfort Pack | Bang and Olufsen Sound system | 20" Gloss black 10 spoke Y design alloys | Sport Plus Pack | Audi Magnetic Ride | Sport exhaust

In [317]:
print(spec_list[104].title)
print(spec_list[104].id)
print(spec_list[104].price)
print(spec_list[104].miles)
print(spec_list[104].url)
print(spec_list[104].descriptionRaw)



Audi R8 5.2 FSI V10 S Tronic RWS (s/s) 2dr

201909092015003
109950
8900
https://www.autotrader.co.uk/classified/advert/201909092015003
Price: £109,950. One of only 100 cars produced for UK. Due to its colour and ultra rare specifications, inc Carbon fibre detailing and Alcantara finishes, this is a true collectors opportunity. PPF covering has helped preserve the car from new., Next MOT due 30/05/2021, First service is not due, Electric windows, Air conditioning, Satellite navigation, Parking aid, MP3 player, CD player, Bluetooth, Leather trim, Heated seats, Height adjustable driver's seat, Height adjustable passenger seat, Sports seats, Alloy wheels, Power steering, Steering wheel rake adjustment, Steering wheel reach adjustment, Cruise control, Traction control, Central locking, Alarm, Immobiliser, Driver's airbags, Side airbags, Passenger airbags, Metallic paint. Grey, 2 owners, £109,950


In [351]:
json_data = {"source": "autotrader", "timestamp":"20191028"}
json_data['ads'] = []

for advert in spec_list:
    json_data['ads'].append({
        'id': advert.id,
        'title': advert.title.strip(),
        'price': advert.price,
        'reg'  : advert.reg_year_month,
        'body_type' : advert.body_type,
        'miles' : advert.miles,
        'cc'    :  advert.cc,
        'tran'  : advert.transmission,
        'fuel'  : advert.fuel,
        'url'   : advert.url,
        'description_raw' : advert.descriptionRaw,
        'other1' : advert.other1,
        'other2' : advert.other2,
        'other3' : advert.other3

        
    })

import json
with open('car_science_autodtrader_20191028.json', 'w') as outfile:
    json.dump(json_data, outfile)
        
       
    